<a href="https://colab.research.google.com/github/rafserran0/ProjetoGeminiStudio/blob/testeCnpj/templates/aistudio_gemini_prompt_chat_b64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [1]:
!pip install -U -q google-generativeai

In [2]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown
from google.colab import userdata

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [4]:
# Configure the client library by providing your API key.
genai.configure(api_key=userdata.get('token'))

### Parse the arguments

In [5]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6ImZvcm1hcyBkZSBjb25zdWx0YXIgbnVtZXJvcyBkZSBDTlBKcyBlIHJldG9ybmFyIG5vbWUgZSBlbmRlcmXDp28gZGFzIGVtcHJlc2FzIn0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6IiMjIENvbnN1bHRhbmRvIENOUEpzIGUgUmV0b3JuYW5kbyBOb21lIGUgRW5kZXJlw6dvOlxuXG5FeGlzdGVtIGRpdmVyc2FzIG1hbmVpcmFzIGRlIGNvbnN1bHRhciBuw7ptZXJvcyBkZSBDTlBKIGUgb2J0ZXIgaW5mb3JtYcOnw7VlcyBjb21vIG5vbWUgZSBlbmRlcmXDp28gZGFzIGVtcHJlc2FzLiBBcXVpIGVzdMOjbyBhbGd1bWFzIG9ww6fDtWVzLCBjb20gdW0gcGFzc28gYSBwYXNzbyBwYXJhIGNhZGEgdW1hOlxuXG4qKjEuIFJlY2VpdGEgRmVkZXJhbDoqKlxuXG4qICoqUGFzc28gMToqKiBBY2Vzc2UgbyBzaXRlIGRhIFJlY2VpdGEgRmVkZXJhbDogaHR0cHM6Ly93d3cuZ292LmJyL3JlY2VpdGFmZWRlcmFsL3B0LWJyXG4qICoqUGFzc28gMjoqKiBOYXZlZ3VlIGF0w6kgYSBzZcOnw6NvIFwiU2VydmnDp29zXCIgZSBzZWxlY2lvbmUgXCJDTlBKXCIuXG4qICoqUGFzc28gMzoqKiBFc2NvbGhhIGEgb3DDp8OjbyBcIkNvbnN1bHRhIFF1YWRybyBTb2NpZXTDoXJpbyBlIFNpdHVhw6fDo28gQ2FkYXN0cmFsXCIuXG4qICoqUGFzc28gNDoqKiBEaWdpdGUgbyBuw7ptZXJvIGRvIENOUEogcXVlIGRlc2VqYSBjb25zdWx0YXIgZSBjbGlxdWUgZW0gXCJDb25zdWx0YXJcIi5cbiogKipQYXNzbyA1OioqIEFzIGluZm9ybWHDp8O1ZXMgZGEgZW1wcmVzYSwgaW5jbHVpbmRvIG5vbWUgZSBlbmRlcmXDp28sIHNlcsOjbyBleGliaWRhcyBuYSB0ZWxhLlxuXG4qKjIuIFNpbnRlZ3JhIChTaXN0ZW1hIEludGVncmFkbyBkZSBJbmZvcm1hw6fDtWVzIHNvYnJlIE9wZXJhw6fDtWVzIEludGVyZXN0YWR1YWlzIGNvbSBNZXJjYWRvcmlhcyBlIFNlcnZpw6dvcyk6KipcblxuKiAqKlBhc3NvIDE6KiogQWNlc3NlIG8gcG9ydGFsIGRvIFNpbnRlZ3JhIGRvIHNldSBlc3RhZG8uIENhZGEgZXN0YWRvIHBvc3N1aSB1bSBwb3J0YWwgcHLDs3ByaW8uXG4qICoqUGFzc28gMjoqKiBMb2NhbGl6ZSBhIHNlw6fDo28gZGUgXCJDb25zdWx0YSBkZSBFbXByZXNhc1wiIG91IHNpbWlsYXIuXG4qICoqUGFzc28gMzoqKiBJbnNpcmEgbyBuw7ptZXJvIGRvIENOUEogZSByZWFsaXplIGEgYnVzY2EuXG4qICoqUGFzc28gNDoqKiBPIHNpc3RlbWEgZXhpYmlyw6EgaW5mb3JtYcOnw7VlcyBjYWRhc3RyYWlzIGRhIGVtcHJlc2EsIGluY2x1aW5kbyBub21lIGUgZW5kZXJlw6dvLlxuXG4qKjMuIEFQSXMgZGUgQ29uc3VsdGEgZGUgQ05QSjoqKlxuXG4qICoqUGFzc28gMToqKiBFc2NvbGhhIHVtYSBBUEkgZGUgY29uc3VsdGEgZGUgQ05QSiwgY29tbyBhIEFQSSBkYSBSZWNlaXRhV1Mgb3UgZGEgS29uZHV0by5cbiogKipQYXNzbyAyOioqIFJlZ2lzdHJlLXNlIG5hIHBsYXRhZm9ybWEgZSBvYnRlbmhhIHVtYSBjaGF2ZSBkZSBhY2Vzc28uXG4qICoqUGFzc28gMzoqKiBJbnRlZ3JlIGEgQVBJIGFvIHNldSBzaXN0ZW1hIG91IGFwbGljYcOnw6NvLlxuKiAqKlBhc3NvIDQ6KiogVXRpbGl6ZSBhIGNoYXZlIGRlIGFjZXNzbyBwYXJhIHJlYWxpemFyIGNvbnN1bHRhcyBwb3IgQ05QSi5cbiogKipQYXNzbyA1OioqIEEgQVBJIHJldG9ybmFyw6EgZGFkb3MgZGEgZW1wcmVzYSBlbSBmb3JtYXRvIEpTT04gb3UgWE1MLCBpbmNsdWluZG8gbm9tZSBlIGVuZGVyZcOnby4gXG5cbioqNC4gUGxhdGFmb3JtYXMgT25saW5lIGRlIENvbnN1bHRhIGRlIEVtcHJlc2FzOioqXG5cbiogKipQYXNzbyAxOioqIEFjZXNzZSB1bWEgcGxhdGFmb3JtYSBkZSBjb25zdWx0YSBkZSBlbXByZXNhcywgY29tbyBhIEVjb25vZGF0YSwgRW1wcmVzw7RtZXRybyBvdSBzaW1pbGFyLlxuKiAqKlBhc3NvIDI6KiogQ3JpZSB1bWEgY29udGEgb3UgdXRpbGl6ZSB1bSBhY2Vzc28gZGUgdGVzdGUsIHNlIGRpc3BvbsOtdmVsLlxuKiAqKlBhc3NvIDM6KiogSW5zaXJhIG8gbsO6bWVybyBkbyBDTlBKIG5hIGJhcnJhIGRlIHBlc3F1aXNhLlxuKiAqKlBhc3NvIDQ6KiogQSBwbGF0YWZvcm1hIGV4aWJpcsOhIHVtIHJlbGF0w7NyaW8gY29tIGluZm9ybWHDp8O1ZXMgZGEgZW1wcmVzYSwgaW5jbHVpbmRvIG5vbWUgZSBlbmRlcmXDp28uIFxuXG4qKk9ic2VydmHDp8O1ZXM6KipcblxuKiBBbGd1bWFzIG9ww6fDtWVzIHBvZGVtIGV4aWdpciBjYWRhc3RybyBvdSBwYWdhbWVudG8gZGUgdGF4YXMuXG4qIEEgcXVhbnRpZGFkZSBkZSBpbmZvcm1hw6fDtWVzIGRpc3BvbsOtdmVpcyBwb2RlIHZhcmlhciBkZXBlbmRlbmRvIGRhIHBsYXRhZm9ybWEgdXRpbGl6YWRhLlxuKiBBbyB1dGlsaXphciBBUElzLCDDqSBuZWNlc3PDoXJpbyB0ZXIgY29uaGVjaW1lbnRvcyBkZSBwcm9ncmFtYcOnw6NvIHBhcmEgaW50ZWdyYXIgYSBBUEkgYW8gc2V1IHNpc3RlbWEuXG4qIFNlbXByZSB2ZXJpZmlxdWUgYSBhdXRlbnRpY2lkYWRlIGRhcyBpbmZvcm1hw6fDtWVzIG9idGlkYXMgZSBhIGZvbnRlIGRhIGNvbnN1bHRhLiBcblxuXG4qKkVzcGVybyBxdWUgZXN0YXMgaW5mb3JtYcOnw7VlcyBzZWphbSDDunRlaXMhKioifSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOiJzZSBldSB0ZSBwYXNzYXIgdW1hIGxpc3RhIGNvbSBudW1lcm9zIGRlIGNucGpzIHZvY2UgY29uc2VndWlyaWEgbWUgcmV0b3JuYXIgYXMgaW5mb3JtYcOnw7VlcyBkZSByYXrDo28gc29jaWFsIGUgZW5kZXJlw6dvIGRhIGVtcHJlc2EgY29ycmVzcG9uZGUgZW0gdW1hIHRhYmVsYT8ifSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjoiQ29tbyB1bSBtb2RlbG8gZGUgbGluZ3VhZ2VtLCBldSBuw6NvIGNvbnNpZ28gYWNlc3NhciBBUElzIGV4dGVybmFzIG91IGJhbmNvcyBkZSBkYWRvcyBlbSB0ZW1wbyByZWFsIHBhcmEgcmVhbGl6YXIgY29uc3VsdGFzIGRlIENOUEouIFBvcnRhbnRvLCBuw6NvIHBvc3NvIGdlcmFyIHVtYSB0YWJlbGEgY29tIGluZm9ybWHDp8O1ZXMgZGUgcmF6w6NvIHNvY2lhbCBlIGVuZGVyZcOnbyBhIHBhcnRpciBkZSB1bWEgbGlzdGEgZGUgQ05QSnMgZGlyZXRhbWVudGUuXG5cbk5vIGVudGFudG8sIHBvc3NvIHRlIGFqdWRhciBkZSBhbGd1bWFzIG1hbmVpcmFzOlxuXG4xLiAqKkluc3RydcOnw7VlcyBEZXRhbGhhZGFzOioqIFBvc3NvIGZvcm5lY2VyIGluc3RydcOnw7VlcyBwYXNzbyBhIHBhc3NvIHNvYnJlIGNvbW8gdm9jw6ogcG9kZSB1c2FyIHVtYSBkYXMgb3DDp8O1ZXMgbWVuY2lvbmFkYXMgYW50ZXJpb3JtZW50ZSAoUmVjZWl0YSBGZWRlcmFsLCBTaW50ZWdyYSwgQVBJcywgcGxhdGFmb3JtYXMgb25saW5lKSBwYXJhIGNvbnN1bHRhciBjYWRhIENOUEogZGEgc3VhIGxpc3RhIGUgb2J0ZXIgYXMgaW5mb3JtYcOnw7VlcyBkZXNlamFkYXMuIFxuMi4gKipDw7NkaWdvIGRlIEV4ZW1wbG86KiogU2Ugdm9jw6ogb3B0YXIgcG9yIHVzYXIgdW1hIEFQSSwgcG9zc28gdGUgYWp1ZGFyIGNvbSBleGVtcGxvcyBkZSBjw7NkaWdvIChQeXRob24sIEphdmFTY3JpcHQsIGV0Yy4pIHBhcmEgZGVtb25zdHJhciBjb21vIGZhemVyIGEgaW50ZWdyYcOnw6NvIGNvbSBhIEFQSSBlIHJlYWxpemFyIGFzIGNvbnN1bHRhcy5cbjMuICoqU3VnZXN0w7VlcyBkZSBGZXJyYW1lbnRhczoqKiBQb3NzbyBzdWdlcmlyIGZlcnJhbWVudGFzIGUgcGxhdGFmb3JtYXMgZXNwZWPDrWZpY2FzIHBhcmEgY29uc3VsdGEgZGUgQ05QSiwgbGV2YW5kbyBlbSBjb25zaWRlcmHDp8OjbyBzdWFzIG5lY2Vzc2lkYWRlcyBlIHByZWZlcsOqbmNpYXMuXG5cbioqUG9yIGZhdm9yLCBtZSBmb3JuZcOnYSBtYWlzIGluZm9ybWHDp8O1ZXMgc29icmUgY29tbyB2b2PDqiBnb3N0YXJpYSBkZSBwcm9jZWRlciBlIHF1YWwgbcOpdG9kbyBkZSBjb25zdWx0YSBwcmVmZXJlLCBwYXJhIHF1ZSBldSBwb3NzYSB0ZSBhdXhpbGlhciBkYSBtZWxob3IgZm9ybWEgcG9zc8OtdmVsLioqIn0seyJyb2xlIjoidXNlciIsInBhcnRzIjoib3DDp8OjbyAyIn0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6IiMjIE9ww6fDo28gMjogVXRpbGl6YW5kbyBBUElzIGRlIENvbnN1bHRhIGRlIENOUEpcblxuUGFyYSBlc3RhIG9ww6fDo28sIHByZWNpc2FyZW1vcyBkZTpcblxuKiAqKkxpbmd1YWdlbSBkZSBQcm9ncmFtYcOnw6NvOioqIFF1YWwgbGluZ3VhZ2VtIGRlIHByb2dyYW1hw6fDo28gdm9jw6ogcHJlZmVyZSB1c2FyPyBQeXRob24gZSBKYXZhU2NyaXB0IHPDo28gb3DDp8O1ZXMgcG9wdWxhcmVzIHBhcmEgdHJhYmFsaGFyIGNvbSBBUElzLlxuKiAqKkFQSSBkZSBDTlBKOioqICBFeGlzdGVtIGRpdmVyc2FzIEFQSXMgZGlzcG9uw612ZWlzLCBjb21vIGEgUmVjZWl0YVdTLCBLb25kdXRvLCBlIG91dHJhcy4gQ2FkYSB1bWEgdGVtIHNldXMgcHLDs3ByaW9zIHBsYW5vcyBlIHByZcOnb3MuIEVzY29saGEgdW1hIHF1ZSBzZSBhZGFwdGUgw6BzIHN1YXMgbmVjZXNzaWRhZGVzLiBcblxuKipFeGVtcGxvIGNvbSBQeXRob24gZSBSZWNlaXRhV1M6KipcblxuYGBgcHl0aG9uXG4jIEltcG9ydGFyIGJpYmxpb3RlY2FzXG5pbXBvcnQgcmVxdWVzdHNcblxuIyBEZWZpbmlyIGEgY2hhdmUgZGUgQVBJIChzdWJzdGl0dWEgcGVsYSBzdWEgY2hhdmUpXG5hcGlfa2V5ID0gXCJTVUFfQ0hBVkVfREVfQVBJXCJcblxuIyBGdW7Dp8OjbyBwYXJhIGNvbnN1bHRhciBDTlBKXG5kZWYgY29uc3VsdGFyX2NucGooY25waik6XG4gICAgdXJsID0gZlwiaHR0cHM6Ly93d3cucmVjZWl0YXdzLmNvbS5ici92MS9jbnBqL3tjbnBqfVwiXG4gICAgaGVhZGVycyA9IHtcIkF1dGhvcml6YXRpb25cIjogZlwiQmVhcmVyIHthcGlfa2V5fVwifVxuICAgIHJlc3BvbnNlID0gcmVxdWVzdHMuZ2V0KHVybCwgaGVhZGVycz1oZWFkZXJzKVxuXG4gICAgaWYgcmVzcG9uc2Uuc3RhdHVzX2NvZGUgPT0gMjAwOlxuICAgICAgICBkYWRvcyA9IHJlc3BvbnNlLmpzb24oKVxuICAgICAgICByYXphb19zb2NpYWwgPSBkYWRvc1snbm9tZSddXG4gICAgICAgIGxvZ3JhZG91cm8gPSBkYWRvc1snbG9ncmFkb3VybyddXG4gICAgICAgIG51bWVybyA9IGRhZG9zWydudW1lcm8nXVxuICAgICAgICBjb21wbGVtZW50byA9IGRhZG9zWydjb21wbGVtZW50byddXG4gICAgICAgIGJhaXJybyA9IGRhZG9zWydiYWlycm8nXVxuICAgICAgICBtdW5pY2lwaW8gPSBkYWRvc1snbXVuaWNpcGlvJ11cbiAgICAgICAgdWYgPSBkYWRvc1sndWYnXVxuICAgICAgICBjZXAgPSBkYWRvc1snY2VwJ11cbiAgICAgICAgXG4gICAgICAgIGVuZGVyZWNvID0gZlwie2xvZ3JhZG91cm99LCB7bnVtZXJvfSAtIHtjb21wbGVtZW50b30sIHtiYWlycm99LCB7bXVuaWNpcGlvfS97dWZ9LCB7Y2VwfVwiXG4gICAgICAgIHJldHVybiByYXphb19zb2NpYWwsIGVuZGVyZWNvXG4gICAgZWxzZTpcbiAgICAgICAgcmV0dXJuIE5vbmUsIE5vbmVcblxuIyBFeGVtcGxvIGRlIHVzb1xuY25waiA9IFwiMDAwMDAwMDAwMDAwMDBcIiAgIyBTdWJzdGl0dWEgcGVsbyBDTlBKIHF1ZSBkZXNlamEgY29uc3VsdGFyXG5yYXphb19zb2NpYWwsIGVuZGVyZWNvID0gY29uc3VsdGFyX2NucGooY25wailcblxuaWYgcmF6YW9fc29jaWFsIGFuZCBlbmRlcmVjbzpcbiAgICBwcmludChmXCJSYXrDo28gU29jaWFsOiB7cmF6YW9fc29jaWFsfVwiKVxuICAgIHByaW50KGZcIkVuZGVyZcOnbzoge2VuZGVyZWNvfVwiKVxuZWxzZTpcbiAgICBwcmludChcIkNOUEogaW52w6FsaWRvIG91IGVycm8gbmEgY29uc3VsdGEuXCIpXG5gYGBcblxuKipFeHBsaWNhw6fDo28gZG8gY8OzZGlnbzoqKlxuXG4xLiBJbXBvcnRhbW9zIGEgYmlibGlvdGVjYSBgcmVxdWVzdHNgIHBhcmEgZmF6ZXIgcmVxdWlzacOnw7VlcyBIVFRQIMOgIEFQSS5cbjIuIERlZmluaW1vcyBhIGBhcGlfa2V5YCBjb20gYSBzdWEgY2hhdmUgZGUgYWNlc3NvIMOgIEFQSSBkYSBSZWNlaXRhV1MuXG4zLiBBIGZ1bsOnw6NvIGBjb25zdWx0YXJfY25waihjbnBqKWAgcmVjZWJlIHVtIG7Dum1lcm8gZGUgQ05QSiBjb21vIGVudHJhZGEuXG40LiBGYXplbW9zIHVtYSByZXF1aXNpw6fDo28gR0VUIMOgIFVSTCBkYSBBUEksIHBhc3NhbmRvIG8gQ05QSiBlIGEgY2hhdmUgZGUgYWNlc3NvIG5vIGhlYWRlci4gXG41LiBTZSBhIHJlc3Bvc3RhIGZvciBiZW0tc3VjZWRpZGEgKHN0YXR1cyBjb2RlIDIwMCksIGV4dHJhw61tb3MgYSByYXrDo28gc29jaWFsIGUgb3MgZGFkb3MgZGUgZW5kZXJlw6dvIGRhIHJlc3Bvc3RhIEpTT04uIFxuNi4gTW9udGFtb3MgYSBzdHJpbmcgZG8gZW5kZXJlw6dvIGNvbXBsZXRvLlxuNy4gUmV0b3JuYW1vcyBhIHJhesOjbyBzb2NpYWwgZSBvIGVuZGVyZcOnby4gXG44LiBObyBleGVtcGxvIGRlIHVzbywgY2hhbWFtb3MgYSBmdW7Dp8OjbyBgY29uc3VsdGFyX2NucGpgIGNvbSB1bSBDTlBKIGUgZXhpYmltb3Mgb3MgcmVzdWx0YWRvcy4gXG5cbioqQWRhcHRhw6fDtWVzOioqXG5cbiogVm9jw6ogcHJlY2lzYXLDoSBzdWJzdGl0dWlyIGBcIlNVQV9DSEFWRV9ERV9BUElcImAgcGVsYSBzdWEgY2hhdmUgZGUgYWNlc3NvIGRhIEFQSSBlc2NvbGhpZGEuXG4qIFBvZGUgc2VyIG5lY2Vzc8OhcmlvIGFqdXN0YXIgbyBjw7NkaWdvIGRlcGVuZGVuZG8gZGEgZXN0cnV0dXJhIGRhIHJlc3Bvc3RhIEpTT04gZGEgQVBJIHF1ZSB2b2PDqiBlc2NvbGhlci5cbiogVm9jw6ogcG9kZSBtb2RpZmljYXIgbyBjw7NkaWdvIHBhcmEgc2FsdmFyIG9zIHJlc3VsdGFkb3MgZW0gdW0gYXJxdWl2bywgYmFuY28gZGUgZGFkb3MsIG91IGV4aWJpciBlbSB1bWEgdGFiZWxhLiBcblxuKipMZW1icmUtc2UgZGUgY29uc3VsdGFyIGEgZG9jdW1lbnRhw6fDo28gZGEgQVBJIGVzY29saGlkYSBwYXJhIG1haXMgZGV0YWxoZXMgc29icmUgYXV0ZW50aWNhw6fDo28sIGVuZHBvaW50cywgbGltaXRlcyBkZSB1c28gZSBvdXRyYXMgaW5mb3JtYcOnw7Vlcy4qKiJ9XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [6]:
contents

[{'role': 'user',
  'parts': 'formas de consultar numeros de CNPJs e retornar nome e endereço das empresas'},
 {'role': 'model',
  'parts': '## Consultando CNPJs e Retornando Nome e Endereço:\n\nExistem diversas maneiras de consultar números de CNPJ e obter informações como nome e endereço das empresas. Aqui estão algumas opções, com um passo a passo para cada uma:\n\n**1. Receita Federal:**\n\n* **Passo 1:** Acesse o site da Receita Federal: https://www.gov.br/receitafederal/pt-br\n* **Passo 2:** Navegue até a seção "Serviços" e selecione "CNPJ".\n* **Passo 3:** Escolha a opção "Consulta Quadro Societário e Situação Cadastral".\n* **Passo 4:** Digite o número do CNPJ que deseja consultar e clique em "Consultar".\n* **Passo 5:** As informações da empresa, incluindo nome e endereço, serão exibidas na tela.\n\n**2. Sintegra (Sistema Integrado de Informações sobre Operações Interestaduais com Mercadorias e Serviços):**\n\n* **Passo 1:** Acesse o portal do Sintegra do seu estado. Cada esta

In [7]:
generation_config

{'temperature': 1,
 'top_p': 0.95,
 'top_k': 0,
 'max_output_tokens': 8192,
 'stop_sequences': []}

In [8]:
safety_settings

[{'category': 'HARM_CATEGORY_HARASSMENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_HATE_SPEECH',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'}]

In [10]:
user_input="qual seria a melhor opção para integrar em um RPA no automation anywhere?"

### Call the API

In [11]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [12]:
display(Markdown(response.text))

Para integrar a consulta de CNPJ em um RPA no Automation Anywhere, a melhor opção seria usar a **API de CNPJ da ReceitaWS**, juntamente com a ação **"Web Service"** do Automation Anywhere.

**Vantagens da API da ReceitaWS para RPA:**

* **Estruturada e Documentada:** A API da ReceitaWS é bem estruturada, com documentação completa e exemplos claros. Isso facilita a integração com o Automation Anywhere.
* **Formato JSON:**  A API retorna os dados em formato JSON, que é fácil de analisar e manipular dentro do Automation Anywhere.
* **Chave de API:** A autenticação via chave de API é segura e fácil de implementar no Automation Anywhere.

**Passo a Passo para Integração com Automation Anywhere:**

1. **Obter Chave de API:** Cadastre-se na ReceitaWS e obtenha sua chave de API.
2. **Criar um Bot no Automation Anywhere:** Crie um novo bot para realizar a consulta de CNPJ.
3. **Adicionar a Ação "Web Service":** Arraste a ação "Web Service" para o fluxo de trabalho do bot.
4. **Configurar a Ação:**
   - **Método:** GET
   - **URL:** `https://www.receitaws.com.br/v1/cnpj/{cnpj}` (Substitua {cnpj} pelo número do CNPJ a ser consultado)
   - **Headers:** Adicione um header com a chave de API:
     - Chave: "Authorization"
     - Valor: "Bearer SUA_CHAVE_DE_API"
5. **Capturar a Resposta:** Utilize a variável de saída da ação "Web Service" para capturar a resposta da API em formato JSON.
6. **Extrair Dados:** Utilize ações de string ou dicionário para extrair a razão social, endereço e outras informações relevantes da resposta JSON.
7. **Utilizar os Dados:** Utilize os dados extraídos para alimentar outras ações do seu RPA, como gravar em um banco de dados, preencher planilhas, etc.

**Exemplo de Bot (Simplificado):**

```
Start
  // Obter CNPJ de alguma fonte (planilha, banco de dados, etc.)
  String cnpj = Get CNPJ;

  // Chamar a API da ReceitaWS
  Web Service:
    Method: GET
    URL: https://www.receitaws.com.br/v1/cnpj/{cnpj}
    Headers:
      Authorization: Bearer SUA_CHAVE_DE_API
    Output: responseJSON

  // Extrair Razão Social
  String razaoSocial = responseJSON["nome"];

  // Extrair Endereço
  String endereco = responseJSON["logradouro"] + ", " + responseJSON["numero"];

  // Exibir os Dados (ou realizar outras ações)
  Message Box:
    Message: "Razão Social: " + razaoSocial + "\nEndereço: " + endereco
End
```

**Recomendações:**

* Consulte a documentação da ação "Web Service" no Automation Anywhere para detalhes sobre configuração e uso.
* Utilize blocos "Try-Catch" para lidar com erros na requisição à API.
* Implemente lógica para lidar com CNPJs inválidos ou inexistentes na base da Receita Federal.

Com a API da ReceitaWS e a ação "Web Service", você terá uma solução robusta e eficiente para consultar CNPJs dentro do seu RPA no Automation Anywhere. 


In [13]:
response.prompt_feedback

In [14]:
response.candidates

[content {
  parts {
    text: "Para integrar a consulta de CNPJ em um RPA no Automation Anywhere, a melhor op\303\247\303\243o seria usar a **API de CNPJ da ReceitaWS**, juntamente com a a\303\247\303\243o **\"Web Service\"** do Automation Anywhere.\n\n**Vantagens da API da ReceitaWS para RPA:**\n\n* **Estruturada e Documentada:** A API da ReceitaWS \303\251 bem estruturada, com documenta\303\247\303\243o completa e exemplos claros. Isso facilita a integra\303\247\303\243o com o Automation Anywhere.\n* **Formato JSON:**  A API retorna os dados em formato JSON, que \303\251 f\303\241cil de analisar e manipular dentro do Automation Anywhere.\n* **Chave de API:** A autentica\303\247\303\243o via chave de API \303\251 segura e f\303\241cil de implementar no Automation Anywhere.\n\n**Passo a Passo para Integra\303\247\303\243o com Automation Anywhere:**\n\n1. **Obter Chave de API:** Cadastre-se na ReceitaWS e obtenha sua chave de API.\n2. **Criar um Bot no Automation Anywhere:** Crie um no